In [1]:
import json, os, itertools
from collections import defaultdict
import gurobipy as gp
import pandas as pd
import numpy as np

BASE_DIR = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"   
edge_fp   = os.path.join(BASE_DIR, "edges.json")
route_fp  = os.path.join(BASE_DIR, "routes_nodes.json")
demand_fp = os.path.join(BASE_DIR, "demand.json")
dept_fp   = os.path.join(BASE_DIR, "dep_time.json")

In [2]:
# 0-b. 고정 파라미터
T        = 12
max_wait = 0
CAPACITY = 3
failed_edges = {"e22","e22r","e30","e30r"}
w1, w2, w3 = 1000, 50, 10
BIG_M = 10**6
SINK  = "SINK"

# 0-c. JSON 로드
with open(edge_fp, encoding="utf-8") as f:
    edges_raw = json.load(f)
edges = {eid: (src, dst, int(tau)) for eid, (src, dst, tau) in edges_raw.items()}

with open(route_fp, encoding="utf-8") as f:
    routes_nodes = json.load(f)

with open(dept_fp, encoding="utf-8") as f:
    dep_time = {tr: int(t) for tr, t in json.load(f).items()}

with open(demand_fp, encoding="utf-8") as f:
    dem_raw = json.load(f)
demand = {tr: [(o, d, float(q)) for o, d, q in lst] for tr, lst in dem_raw.items()}

trains = list(routes_nodes)
nodes  = {n for _, (s, d, _) in edges.items() for n in (s, d)}
nodes.add(SINK)

# ----- 시간-확장 아크 ---------------------------------------------------
arc_list = []                       # (from,to,eid,tau,t0,t1)
for eid, (src, dst, tau) in edges.items():
    if eid in failed_edges: continue
    for t in range(T + 1 - tau):
        arc_list.append((f"{src}^{t}", f"{dst}^{t+tau}", eid, tau, t, t+tau))

for n in nodes - {SINK}:
    for w in range(1, max_wait+1):
        for t in range(T + 1 - w):
            arc_list.append((f"{n}^{t}", f"{n}^{t+w}", f"w_{n}_{w}", w, t, t+w))

for n in nodes - {SINK}:
    for t in range(T+1):
        arc_list.append((f"{n}^{t}", f"{SINK}^{t}", f"dummy_{n}", 0, t, t))

arc_idx = {info: i for i, info in enumerate(arc_list)}

# ───────────────── 캐싱: arc → 노드·시각 별 in/out ─────────────────
out_arcs = defaultdict(list)   # (node,t) → [arcIdx…]
in_arcs  = defaultdict(list)   # (node,t) → [arcIdx…]
node_in_arcs  = defaultdict(list)   # 노드 방문용
node_out_arcs = defaultdict(list)

for k, (fr, to, eid, _, t0, t1) in enumerate(arc_list):
    n_fr, tt_fr = fr.split("^"); tt_fr = int(tt_fr)
    n_to, tt_to = to.split("^"); tt_to = int(tt_to)

    out_arcs[(n_fr, tt_fr)].append(k)
    in_arcs [(n_to, tt_to)].append(k)

    if not eid.startswith(("w_", "dummy")):      # s[tr,n] 계산용
        node_out_arcs[n_fr].append(k)
        node_in_arcs [n_to].append(k)


# ----- 예정 도착시각 ----------------------------------------------------
sched = {}
for tr, path in routes_nodes.items():
    t, arr = 0, {path[0]: dep_time[tr]}
    for u, v in zip(path[:-1], path[1:]):
        eid = next(e for e, (s, d, _) in edges.items() if s == u and d == v)
        t += edges[eid][2]; arr[v] = dep_time[tr] + t
    sched[tr] = arr

q_r = {tr: sum(q for *_, q in demand[tr]) for tr in trains}

In [3]:
def build_base_model():
    m = gp.Model(); m.Params.OutputFlag = 1
    nA, nT = len(arc_list), len(trains)

    x = m.addVars(nA, nT, vtype=gp.GRB.BINARY, name="x")
    h = m.addVars(trains, vtype=gp.GRB.BINARY, name="h")

    y = {(tr, n): m.addVar(vtype=gp.GRB.BINARY) for tr in trains for n in routes_nodes[tr][1:]}
    s = {(tr, n): m.addVar(vtype=gp.GRB.BINARY) for tr in trains for n in nodes - {SINK}}
    z = {(tr,o,d): m.addVar(vtype=gp.GRB.BINARY)
         for tr in trains for (o,d,_) in demand[tr]}
    delta = {(tr,o,d): m.addVar(lb=0)
             for tr in trains for (o,d,_) in demand[tr]}
    t_arr = {(tr, n): m.addVar(lb=0, ub=T, vtype=gp.GRB.INTEGER)
             for tr in trains for n in nodes - {SINK}}

    # (2) 출발-flow
    for tr_i, tr in enumerate(trains):
        r_o, t_dep = routes_nodes[tr][0], dep_time[tr]
        idx_out = out_arcs[(r_o, t_dep)]          # ← 캐시
        m.addConstr(x.sum(idx_out, tr_i) == h[tr])

    # (5) 노드-시간 보존
    for tr_i, tr in enumerate(trains):
        r_o, t_dep = routes_nodes[tr][0], dep_time[tr]
        for n in nodes - {SINK}:
            for t in range(T+1):
                inflow  = x.sum(in_arcs[(n, t)],  tr_i)
                outflow = x.sum(out_arcs[(n, t)], tr_i)
                if (n == r_o) and (t == t_dep):
                    m.addConstr(outflow - inflow == h[tr])
                else:
                    m.addConstr(inflow == outflow)

    # terminal & dummy
    for tr_i, tr in enumerate(trains):
        cand = routes_nodes[tr][1:]
        m.addConstr(gp.quicksum(y[tr, n] for n in cand) == h[tr])
        for n in cand:
            idx_dum = [arc_idx[a] for a in arc_list if a[2] == f"dummy_{n}"]
            m.addConstr(x.sum(idx_dum, tr_i) == y[tr, n])

    # s (visit)
    for (tr, n), var in s.items():
        tr_i = trains.index(tr)
        idx = node_out_arcs[n] + node_in_arcs[n]   # 두 리스트 모두 이용
        flow = x.sum(idx, tr_i)
        m.addConstr(flow >= var)
        m.addConstr(flow <= BIG_M * var)
        
    # z-logic & delta
    for tr in trains:
        for (o, d, q) in demand[tr]:
            m.addConstr(z[tr, o, d] <= s[tr, o])
            m.addConstr(z[tr, o, d] <= s[tr, d])
            m.addConstr(z[tr, o, d] >= s[tr, o] + s[tr, d] - 1)
            m.addConstr(z[tr, o, d] <= h[tr])

    for tr_i, tr in enumerate(trains):
        for k, (fr, to, eid, *_ ) in enumerate(arc_list):
            if eid.startswith(("w_", "dummy")): continue
            n_to, tt = to.split("^"); tt = int(tt)
            m.addConstr(t_arr[tr, n_to] >= tt - BIG_M * (1 - x[k, tr_i]))
            m.addConstr(t_arr[tr, n_to] <= tt + BIG_M * (1 - x[k, tr_i]))
        for (o, d, q) in demand[tr]:
            sched_t = sched[tr].get(d, T)
            m.addConstr(delta[tr, o, d] >= t_arr[tr, d] - sched_t - BIG_M * (1 - z[tr, o, d]))
            m.addConstr(delta[tr, o, d] <= BIG_M * z[tr, o, d])

    obj  = gp.quicksum(w1 * q_r[tr] * (1 - h[tr]) for tr in trains)
    obj += gp.quicksum(w2 * q * (1 - z[tr, o, d]) for tr in trains for (o, d, q) in demand[tr])
    obj += gp.quicksum(w3 * q * delta[tr, o, d] * z[tr, o, d]
                       for tr in trains for (o, d, q) in demand[tr])
    m.setObjective(obj)
    return m, x, h, delta, z

# cap_map: (eid,tt) → [arcIdx …]
cap_map = defaultdict(list)
for k, (fr, to, eid, _, t0, t1) in enumerate(arc_list):
    if eid.startswith(("w_", "dummy")): continue
    for tt in range(t0, t1):
        cap_map[(eid, tt)].append(k)

In [4]:
working_set, sol_prev = set(), None
MAX_ITER, EPS, SHOW = 40, 1e-6, 30

for it in range(MAX_ITER):
    base, xvar, hvar, delta, z = build_base_model()

    constr_refs = {}
    for (eid, tt) in working_set:
        expr = gp.quicksum(xvar[idx, ti] for idx in cap_map[(eid, tt)]
                           for ti in range(len(trains)))
        constr_refs[(eid, tt)] = base.addConstr(expr <= CAPACITY,
                                               name=f"cap_{eid}_{tt}")

    if sol_prev:
        for (idx, ti), v in sol_prev.items():
            xvar[idx, ti].Start = v

    base.optimize()
    if base.Status != gp.GRB.OPTIMAL:
        raise RuntimeError("non-optimal")

    x_val = {(idx, ti): round(xvar[idx, ti].X)
         for idx in range(len(arc_list))
         for ti  in range(len(trains))}
    
    # ③-c violation search
    x_mat = np.zeros((len(arc_list), len(trains)), dtype=np.uint8)
    for (idx, ti), v in x_val.items():      # x_val 은 아래에서 다시 만듦
        if v: x_mat[idx, ti] = 1
    cap_keys   = list(cap_map)                         # [(eid,tt), …]
    cap_idxarr = [np.fromiter(cap_map[k], dtype=int)   # each → np.array
                for k in cap_keys]
    viol = set()
    for k, idx_arr in zip(cap_keys, cap_idxarr):
        if x_mat[idx_arr].sum() > CAPACITY + EPS:
            viol.add(k)

    # ③-d negative dual search
    neg_pi = set()
    lp = base.relax(); lp.optimize()
    if lp.Status == gp.GRB.OPTIMAL:
        for key, c in constr_refs.items():
            if c.Pi < -EPS: neg_pi.add(key)
            
    # ③-e working-set update
    updated = False
    for k in viol:
        if k not in working_set: working_set.add(k); updated = True
    for k in neg_pi:
        if k in working_set: working_set.remove(k); updated = True

    ws = sorted(working_set)
    print(f"[Iter {it}] |W|={len(ws)}  preview={ws[:SHOW]}")
    print(f"          add={len(viol)} drop={len(neg_pi)}")
    sol_prev = x_val
    if not updated:
        print("↳ converged"); break
else:
    print("⚠ MAX_ITER reached")

Set parameter Username
Set parameter LicenseID to value 2637066
Academic license - for non-commercial use only - expires 2026-03-16
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 593392 rows, 357549 columns and 2449221 nonzeros
Model fingerprint: 0x4e1bdb2c
Model has 1867 quadratic objective terms
Variable types: 1867 continuous, 355682 integer (347387 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e+02, 1e+06]
  QObjective range [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 8.340259e+07
Presolve removed 538188 rows and 324668 columns
Presolve time: 2.32s
Presolved: 55871 rows, 33548 columns, 211462 nonzeros
Variable types: 0 

In [5]:
rows = []
for tr_i, tr in enumerate(trains):
    if hvar[tr].X < 0.5:
        rows.append([tr, '-', '-', '-', '-', 'Cancelled'])
        continue
    legs = [(int(arc_list[k][0].split("^")[1]),
             arc_list[k][0].split("^")[0],
             arc_list[k][1].split("^")[0],
             int(arc_list[k][1].split("^")[1]),
             arc_list[k][2])
            for k in range(len(arc_list))
            if xvar[k, tr_i].X > 0.5 and not arc_list[k][2].startswith(("w_", "dummy"))]
    legs.sort(key=lambda v: v[0])
    for dep, frm, to, arr, eid in legs:
        rows.append([tr, frm, dep, to, arr, eid])

pd.set_option('display.max_rows', None)
df = pd.DataFrame(rows, columns=["Train", "From", "Dep", "To", "Arr", "Edge"])
print("\n=== Timetable ==="); display(df.head(3)) 

# ───────────────────────── Train status summary
status_rows = []
for tr_i, tr in enumerate(trains):
    if hvar[tr].X < 0.5:
        status = "Cancelled"
    else:
        delayed = any(delta[tr, o, d].X > 1e-6 for (o, d, _) in demand[tr])
        unmet   = any(z[tr, o, d].X < 0.5 for (o, d, _) in demand[tr])
        if delayed:
            status = "Delayed"
        elif unmet:
            status = "Truncated/Rerouted"
        else:
            status = "On-time & complete"
    status_rows.append([tr, status])

df_stat = pd.DataFrame(status_rows, columns=["Train", "Status"])
print("\n=== Train status summary ===")
display(df_stat.head(3))


=== Timetable ===


,Train,From,Dep,To,Arr,Edge
0,경부고속철도1_1,n4,0,n3,1,e4
1,경부고속철도1_1,n3,1,n1,2,e3
2,경부고속철도1_1,n1,2,n2,3,e1



=== Train status summary ===


,Train,Status
0,경부고속철도1_1,Truncated/Rerouted
1,경부고속철도1_2,Truncated/Rerouted
2,경부고속철도1_3,Truncated/Rerouted
